### Import headers

In [3]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import sys
sys.path.append("..")

import argparse
import os
import time
from dataclasses import dataclass

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import cifar10, mnist
from models import FullyConnectedNet, LeNet
from src import hessians, lanczos, regularization

device = "cuda" if torch.cuda.is_available() else "cpu"
root = ".."

### Define a wrapper function

This makes my life more easier

In [4]:
def compute_eigvals(data, target, path:int, network: str, loss:str, alpha: float, num_eigval: int=500):
    # Network configuration
    if network == "LeNet":
        net = LeNet().to(device)
    else:
        net = FullyConnectedNet(28 * 28, 8, 10, 3, 0.1).to(device)
        flatten = True
    net_name = net.__class__.__name__

    if device == "cuda":
        net = torch.nn.DataParallel(net)
        cudnn.benchmark = True

    # Load checkpoint.
    assert os.path.isfile(
        f"{root}/checkpoints/{path}/{net_name}/{loss}/ckpt_{alpha}.pth"
    ), "Error: no checkpoint file found!: "+ f"checkpoints/{path}/{net_name}/{loss}/ckpt_{alpha}.pth"
    checkpoint = torch.load(f"{root}/checkpoints/{path}/{net_name}/{loss}/ckpt_{alpha}.pth")
    net.load_state_dict(checkpoint["net"])
    assert (
        checkpoint["alpha"] == alpha
    ), "Error: alpha is not equal to checkpoint value!"
    assert (
        checkpoint["criterion"] == loss
    ), "Error: loss is not equal to checkpoint value!"

    # Loss configuration
    if loss == "cross_entropy":
        criterion = regularization.RegularizedLoss(
            net, nn.CrossEntropyLoss(), alpha
        )
        one_hot = False
    else:
        criterion = regularization.RegularizedLoss(net, nn.MSELoss(), alpha)
        one_hot = True
        
    loss = criterion(net(data.to(device)), target.to(device))

    start = time.time()
    if device == "cuda":
        eigvals_lanczos, _ = lanczos.lanczos(
            loss,
            net,
            num_eigenthings=num_eigval,
            tol=0,
            use_gpu=True,
        )
    else:
        eigvals_lanczos, _ = lanczos.lanczos(
            loss,
            net,
            num_eigenthings=num_eigval,
            tol=0,
        )

    return np.sum(eigvals_lanczos < -1e-8)

### Call dataloaders

In [5]:
#LeNet, CIFAR10
#FCN, MNIST
#loss: cross_entropy, mse
#Models: 1-10
#alpha 0.00 - 0.25, 0.01

num_eigval = 400

# Data
batch_size = 512
num_workers = 2

data_loader_mnist = mnist.MNISTDataLoader(batch_size, num_workers, one_hot=False, flatten=True)
_, _, test_loader_mnist_ce = data_loader_mnist.get_data_loaders()

data_loader_mnist = mnist.MNISTDataLoader(batch_size, num_workers, one_hot=True, flatten=True)
_, _, test_loader_mnist_mse = data_loader_mnist.get_data_loaders()

data_loader_cifar10 = cifar10.CIFAR10DataLoader(batch_size, num_workers, one_hot=False)
_, _, test_loader_cifar10_ce = data_loader_cifar10.get_data_loaders()

data_loader_cifar10 = cifar10.CIFAR10DataLoader(batch_size, num_workers, one_hot=True)
_, _, test_loader_cifar10_mse = data_loader_cifar10.get_data_loaders()

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [18]:
def recursive_computation(data_loader, network, loss):
    ratio_list = []
    for alpha in range(0, 26):
        alpha /= 100.
        num_negative = []
        for path  in range(1,101): #26
            for batch in range(1):
                data, target = next(iter(data_loader))
                result = compute_eigvals(data, target, path, network, loss, alpha, num_eigval)
                num_negative.append(result / num_eigval)
                print(f"{path = }, {batch = }, {result = }")
        average = sum(num_negative)/len(num_negative)*100
        print(alpha, f"{average = :.2f} %")
        ratio_list.append(average)
    return ratio_list

In [19]:
ratio_list_mnist_ce = recursive_computation(test_loader_mnist_mse, "FCN", "mse")

path = 1, batch = 0, result = 15
path = 2, batch = 0, result = 13
path = 3, batch = 0, result = 13
path = 4, batch = 0, result = 14
path = 5, batch = 0, result = 14
path = 6, batch = 0, result = 12
path = 7, batch = 0, result = 13
path = 8, batch = 0, result = 12
path = 9, batch = 0, result = 13
path = 10, batch = 0, result = 13
path = 11, batch = 0, result = 13
path = 12, batch = 0, result = 11
path = 13, batch = 0, result = 12
path = 14, batch = 0, result = 10
path = 15, batch = 0, result = 13
path = 16, batch = 0, result = 10
path = 17, batch = 0, result = 14
path = 18, batch = 0, result = 13
path = 19, batch = 0, result = 13
path = 20, batch = 0, result = 13
path = 21, batch = 0, result = 11
path = 22, batch = 0, result = 12
path = 23, batch = 0, result = 12
path = 24, batch = 0, result = 13
path = 25, batch = 0, result = 13
path = 26, batch = 0, result = 12
path = 27, batch = 0, result = 12
path = 28, batch = 0, result = 14
path = 29, batch = 0, result = 14
path = 30, batch = 0, r

path = 42, batch = 0, result = 12
path = 43, batch = 0, result = 9
path = 44, batch = 0, result = 14
path = 45, batch = 0, result = 12
path = 46, batch = 0, result = 12
path = 47, batch = 0, result = 12
path = 48, batch = 0, result = 13
path = 49, batch = 0, result = 11
path = 50, batch = 0, result = 12
path = 51, batch = 0, result = 14
path = 52, batch = 0, result = 10
path = 53, batch = 0, result = 15
path = 54, batch = 0, result = 13
path = 55, batch = 0, result = 9
path = 56, batch = 0, result = 8
path = 57, batch = 0, result = 10
path = 58, batch = 0, result = 14
path = 59, batch = 0, result = 13
path = 60, batch = 0, result = 12
path = 61, batch = 0, result = 11
path = 62, batch = 0, result = 15
path = 63, batch = 0, result = 14
path = 64, batch = 0, result = 10
path = 65, batch = 0, result = 14
path = 66, batch = 0, result = 12
path = 67, batch = 0, result = 12
path = 68, batch = 0, result = 13
path = 69, batch = 0, result = 11
path = 70, batch = 0, result = 13
path = 71, batch 

path = 88, batch = 0, result = 1
path = 89, batch = 0, result = 3
path = 90, batch = 0, result = 2
path = 91, batch = 0, result = 2
path = 92, batch = 0, result = 2
path = 93, batch = 0, result = 4
path = 94, batch = 0, result = 1
path = 95, batch = 0, result = 2
path = 96, batch = 0, result = 0
path = 97, batch = 0, result = 0
path = 98, batch = 0, result = 2
path = 99, batch = 0, result = 3
path = 100, batch = 0, result = 1
0.04 average = 0.52 %
path = 1, batch = 0, result = 0
path = 2, batch = 0, result = 0
path = 3, batch = 0, result = 0
path = 4, batch = 0, result = 0
path = 5, batch = 0, result = 0
path = 6, batch = 0, result = 0
path = 7, batch = 0, result = 0
path = 8, batch = 0, result = 0
path = 9, batch = 0, result = 0
path = 10, batch = 0, result = 0
path = 11, batch = 0, result = 0
path = 12, batch = 0, result = 0
path = 13, batch = 0, result = 0
path = 14, batch = 0, result = 0
path = 15, batch = 0, result = 0
path = 16, batch = 0, result = 0
path = 17, batch = 0, result 

path = 35, batch = 0, result = 0
path = 36, batch = 0, result = 0
path = 37, batch = 0, result = 0
path = 38, batch = 0, result = 0
path = 39, batch = 0, result = 0
path = 40, batch = 0, result = 0
path = 41, batch = 0, result = 0
path = 42, batch = 0, result = 0
path = 43, batch = 0, result = 0
path = 44, batch = 0, result = 0
path = 45, batch = 0, result = 0
path = 46, batch = 0, result = 0
path = 47, batch = 0, result = 0
path = 48, batch = 0, result = 0
path = 49, batch = 0, result = 0
path = 50, batch = 0, result = 0
path = 51, batch = 0, result = 0
path = 52, batch = 0, result = 0
path = 53, batch = 0, result = 0
path = 54, batch = 0, result = 0
path = 55, batch = 0, result = 0
path = 56, batch = 0, result = 0
path = 57, batch = 0, result = 0
path = 58, batch = 0, result = 0
path = 59, batch = 0, result = 0
path = 60, batch = 0, result = 0
path = 61, batch = 0, result = 0
path = 62, batch = 0, result = 0
path = 63, batch = 0, result = 0
path = 64, batch = 0, result = 0
path = 65,

ArpackError: ARPACK error 3: No shifts could be applied during a cycle of the Implicitly restarted Arnoldi iteration. One possibility is to increase the size of NCV relative to NEV. 

In [8]:
ratio_list_mnist_mse = recursive_computation(test_loader_mnist_ce, "FCN", "cross_entropy")

path = 1, batch = 0, result = 79


KeyboardInterrupt: 

In [ ]:
import matplotlib as plt

W_reg = np.arange(0, 0.26, 0.01).tolist()

plt.subplot(1,2,1)
plt.plot(W_reg, ratio_list_mnist_ce, label="MNIST/CE")
plt.xlabel('Regulaization Weight')
plt.ylabel('Negative Eigen Value Rate')
plt.legend()
plt.subplot(1,2,2)
plt.plot(W_reg, ratio_list_mnist_mse, label="MNIST/MSE")
plt.xlabel('Regulaization Weight')
plt.ylabel('Negative Eigen Value Rate')
plt.legend()

plt.show()